In [ ]:
from google.cloud import bigquery
import os
from datetime import datetime

# Defina a variável de ambiente para o caminho da chave .json
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = r"C:\Users\geraldo.junior\DEV\Marketing_IA\credenciais\big_query_key_geraldo-junior.json"

# Crie um cliente do BigQuery
client = bigquery.Client()

# Defina o conjunto de dados de origem e de backup
source_dataset = "meta-vc-contabil.marketing_db"
backup_dataset = "meta-vc-contabil.marketing_db_backup_BY_VSCODE"

# Liste todas as tabelas no conjunto de dados de origem
tables = client.list_tables(source_dataset)

# Função para obter a data da última modificação de uma tabela
def get_last_modified_time(table_id):
    table = client.get_table(table_id)
    return table.modified

# Função para copiar uma tabela para o conjunto de dados de backup
def backup_table_function(table):
    table_id = table.table_id  # Nome da tabela
    source_table = f"{source_dataset}.{table_id}"
    backup_table_id = f"{backup_dataset}.{table_id}"

    # Configuração do job de cópia
    job = client.copy_table(source_table, backup_table_id)
    job.result()  # Aguarda a conclusão do job

    print(f"Tabela {source_table} copiada para {backup_table_id}.")

# Defina o diretório para salvar os arquivos de log
log_directory = r"C:\Users\geraldo.junior\DEV\marketing_db\historicoExecucao"

# Verifique se o diretório existe; se não, crie-o
if not os.path.exists(log_directory):
    os.makedirs(log_directory)

# Crie um arquivo TXT para registrar as informações, com a data de execução no nome do arquivo
execution_date = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
log_file = os.path.join(log_directory, f"backup_log_{execution_date}.txt")

# Variável para controlar se alguma tabela foi alterada
tabelas_alteradas = False

with open(log_file, "w") as f:
    f.write(f"Data de execução: {datetime.now()}\n\n")

# Faça o backup apenas das tabelas que foram alteradas
for table in tables:
    table_id = table.table_id
    source_table = f"{source_dataset}.{table_id}"
    backup_table_id = f"{backup_dataset}.{table_id}"

    try:
        source_last_modified = get_last_modified_time(source_table)
        backup_last_modified = get_last_modified_time(backup_table_id)

        if source_last_modified > backup_last_modified:
            tabelas_alteradas = True  # Marca que houve alteração
            backup_table_function(table)
            # Obtenha o número de linhas alteradas
            query = f"SELECT COUNT(*) as row_count FROM {source_table}"
            query_job = client.query(query)
            row_count = list(query_job)[0].row_count

            with open(log_file, "a") as f:
                f.write(f"Nome da tabela: {table_id}\n")
                f.write(f"Quantidade de linhas alteradas: {row_count}\n\n")
    except Exception as e:
        # Se a tabela de backup não existir, crie uma cópia
        tabelas_alteradas = True  # Marca que houve alteração
        backup_table_function(table)
        with open(log_file, "a") as f:
            f.write(f"Nome da tabela: {table_id}\n")
            f.write(f"Quantidade de linhas alteradas: Tabela copiada pela primeira vez.\n\n")

# Se nenhuma tabela foi alterada, adicione a mensagem ao arquivo de log
if not tabelas_alteradas:
    with open(log_file, "a") as f:
        f.write("Nenhuma tabela atualizada.\n")

print(f"Backup concluído! Arquivo de log salvo em: {log_file}")